In [4]:
parent_dir = joinpath(@__DIR__, "..")
include(joinpath(parent_dir, "helper_split.jl"))
include(joinpath(parent_dir, "algorithms.jl"))
include(joinpath(parent_dir, "helper_tt.jl"))

using Distributions,ForwardDiff,LinearAlgebra,Plots

Set up the target, the space transform, and the speed function:

In [ ]:
dim = 2

s(x)  = (1 + norm(x)^2)^((dim + 1)/2)
∇s(x) = (dim + 1) * (1 + norm(x)^2)^((dim - 1)/2) * x
H_inv(x) = x / ((1 + norm(x)^2)^(1/2))
H(y) = y / ((1-norm(y)^2)^(1/2))

# nu = 15
# p = dim
# ∇U(x) = ((nu + p) / nu) * x / (1 + norm(x)^2/nu)
# Hessian_bound_alpha = diagm((nu + p) * ones(dim))
# U(x) = 0.5 * (nu + p) * log((1 + nu^(-1) * transpose(x) * x))

U(x)  = 0.5 * norm(x)^2
∇U(x) = x
norm_const = 1 / sqrt(2 * pi)
Hessian_bound_alpha = diagm((dim + 1)^2 * ones(dim))
target(x) = norm_const * exp(-U(x))
∇U_s(x) = ∇U(x) - ∇s(x)/s(x)
unnorm_target_s(x) = s(x) * exp(-U(x))


Plot the base distribution for the time-changed process

In [ ]:
x = range(-4, stop = 4, length = 1000)
y = range(-4, stop = 4, length = 1000)
contourf(x, y, (x, y) -> (unnorm_target_s([x,y])),color=:plasma,ratio=:equal)

Produce the first plot: time-changed ZZP with speed s

In [ ]:
T = 1 * 10^2
delta = 1e-2
x_init = zeros(dim)
v_init = rand((-1,1),dim)
skele = ZigZag(∇U_s, Hessian_bound_alpha, T, x_init,v_init)

chain_tt = approximate_timechanged_skele(skele, s, delta)  # this gives skeleton
pos1 = [x.position[1] for (_,x) in enumerate(chain_tt)]
pos2 = [x.position[2] for (_,x) in enumerate(chain_tt)]
pos = [x.position for (_,x) in enumerate(chain_tt)]
speeds = [norm(pos[i+1]-pos[i])/(chain_tt[i+1].time - chain_tt[i].time) for i = 1 : length(chain_tt)-1]
push!(speeds, speeds[end])
# speeds = [s(x.position) for (_,x) in enumerate(chain_tt)]
p_tt = plot(pos1, pos2, label = "", 
                # lc = "red",
                lw = 1.5, grid=:none, xlabel = L"x_1", ylabel = L"x_2",
                # color=:darktest,
                line_z = log.(speeds),
                clim = (0.2,4.7),
                aspect_ratio=:equal,
                # xlims = [-4,4], 
                ylims = [-4,4]
                )
display(p_tt) 
# StatsPlots.savefig("timechanged.pdf")

Now produce the second plot: apply the corresponding space transformation to the time-changed ZZP

In [ ]:
transformed_pos = [H_inv(x.position) for (_,x) in enumerate(chain_tt)]
speeds_space = [norm(transformed_pos[i+1]-transformed_pos[i]) / (chain_tt[i+1].time - chain_tt[i].time) for i = 1 : length(transformed_pos)-1]
push!(speeds_space, speeds_space[end])

pos_transf_1 = [x[1] for (_,x) in enumerate(transformed_pos)]
pos_transf_2 = [x[2] for (_,x) in enumerate(transformed_pos)]
p_space = plot(pos_transf_1, pos_transf_2, label = "", 
                # markershape=:x,
                lw = 1.5, grid=:none, xlabel = L"x_1", ylabel = L"x_2",
                # color=:darktest,
                line_z = log.(speeds_space),
                clim = (0.2,4.7),
                aspect_ratio=:equal,
                )
theta = 0:0.01:2π
p_space = plot!(cos.(theta),sin.(theta), lc="green", lw = 3, label="",aspect_ratio=:equal)
display(p_space)
# StatsPlots.savefig("timechanged_circle.pdf")

Check what is the minimum and maximum speed of the process in the circle

In [ ]:
m = minimum(speeds_space)
M = maximum(speeds_space)
println("The minimum speed is $m, while the maximum speed is $M")

Plot the base distribution for the space-transformed process

In [ ]:
using GR 
ρ = LinRange(0., 0.999, 200) 
θ = LinRange(0., 2π, 360) 
polar_function(rh, th) = s(H([rh*cos(th),rh*sin(th)])) * target(H([rh*cos(th),rh*sin(th)]))
polarheatmap(polar_function.(ρ,θ'))

Obtain the base process in the circle using a splitting scheme approximation of Zig-Zag

In [ ]:
δ = 2e-3
T = 3 * 10^1
N = Int(ceil(T/δ))
x_init = zeros(dim)
v_init = rand((-1,1),dim)
# ∇U_tilde(y) = y * ( norm(y)^2 / ((1-norm(y)^2)^2) - dim / (1-norm(y)^2))
∇U_tilde(y) = y / ((1-norm(y)^2)^2) * (norm(y)^2 - (dim+1) * (1-norm(y)^2))
skele_split = splitting_zzs_DBD(∇U_tilde, δ, N,x_init,v_init)
pos1 = [sk.position[1] for (_,sk) in enumerate(skele_split)]
pos2 = [sk.position[2] for (_,sk) in enumerate(skele_split)]
plot(pos1,pos2,label="", lw = 1.5, lc = "black",grid=:none, 
    line_z = log.(ones(length(pos1)) * sqrt(dim)),
    clim = (0.2,4.7),
    xlabel = L"x_1", ylabel = L"x_2",)
theta = 0:0.01:2π
display(plot!(cos.(theta),sin.(theta), lc="green", lw = 3, label="",aspect_ratio=:equal))
# StatsPlots.savefig("spacetransf_circle.pdf")

Now apply a space transformation to obtain the process on $\mathbb{R}^2$

In [ ]:
transformed_pos_split = [H(x.position) for (_,x) in enumerate(skele_split)]
speeds_space_split = [norm(transformed_pos_split[i+1]-transformed_pos_split[i]) / (skele_split[i+1].time - skele_split[i].time) for i = 1 : length(transformed_pos_split)-1]
push!(speeds_space_split, speeds_space_split[end])

pos_transf_1 = [x[1] for (_,x) in enumerate(transformed_pos_split)]
pos_transf_2 = [x[2] for (_,x) in enumerate(transformed_pos_split)]
p_space = plot(pos_transf_1, pos_transf_2, label = "", 
                # markershape=:x,
                lw = 1.5, grid=:none, xlabel = L"x_1", ylabel = L"x_2",
                # color=:darktest,
                line_z = log.(speeds_space_split),
                clim = (0.2,4.7),
                aspect_ratio=:equal,
                # xlims = [-4,4], 
                ylims = [-4,4]
                )
display(p_space)
# StatsPlots.savefig("spacetransf.pdf")